<a href="https://colab.research.google.com/github/Sudeeppp-Mishra/LeafLens/blob/main/LeafLens_Train_on_DataSets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LeafLens: AI-Based Plant Disease Detection

LeafLens classifies plant leaf images into healthy or diseased categories.
We use a pretrained ResNet18 model (transfer learning) for accuracy and faster training.

**Libraries used:**
- PIL / OpenCV / NumPy: Image loading and preprocessing
- PyTorch / Torchvision: Model training and evaluation
- Matplotlib: Visualization of metrics
- PySide6 / pyttsx3 / gTTS: GUI + audio output (later integration)

**Dataset:** PlantVillage (Image Classification)  
**Platform:** Google Colab (GPU)

## Why GPU?

Deep learning models require heavy computation.  
Using GPU in Colab speeds up training significantly.

## Mount Google Drive

Accessing PlantVillage dataset stored in Google Drive.  
This avoids repeated uploads and keeps large datasets organized.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Required Libraries

- torch, nn, optim: Core deep learning
- torchvision: Pretrained models and transforms
- cv2: OpenCV for image preprocessing
- PIL: For PyTorch compatibility
- matplotlib, numpy: Visualization & arrays
- os: File management

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

import matplotlib.pyplot as plt
import numpy as np
import os

import cv2
from PIL import Image

## Check GPU

We confirm if GPU is available.  
Training on GPU is much faster than CPU.

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


## Dataset Path

Set path to PlantVillage dataset on Google Drive.
Each subfolder corresponds to a class label.

In [5]:
data_dir = "/content/drive/MyDrive/datasets/PlantVillage"
print("Classes found:", os.listdir(data_dir))

Classes found: ['Tomato_healthy', 'PlantVillage', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato_Leaf_Mold', 'Potato___Early_blight', 'Tomato_Septoria_leaf_spot', 'Potato___Late_blight', 'Tomato_Early_blight', 'Tomato__Target_Spot', 'Pepper__bell___healthy', 'Tomato__Tomato_mosaic_virus', 'Tomato_Bacterial_spot', 'Potato___healthy', 'Pepper__bell___Bacterial_spot', 'Tomato_Late_blight']


## Image Preprocessing & Augmentation

**Why:**  
- Resize images to 224x224 (ResNet input requirement)  
- Normalize pixel values for stable training  
- Data augmentation (rotation, flip) improves generalization  
- OpenCV is used for preprocessing (denoising, resizing)  
- PIL is used to convert images into format compatible with PyTorch transforms

In [6]:
from torch.utils.data import Dataset

class PlantVillageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}

        # Load image paths and labels
        for cls in self.classes:
            cls_folder = os.path.join(root_dir, cls)
            for img_name in os.listdir(cls_folder):
                img_path = os.path.join(cls_folder, img_name)
                self.images.append(img_path)
                self.labels.append(self.class_to_idx[cls])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]

        # Read image with OpenCV
        img = cv2.imread(img_path)
        if img is None:
            # Handle unreadable image
            img = np.zeros((224,224,3), dtype=np.uint8)

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224,224))
        img = cv2.GaussianBlur(img, (3,3), 0)  # optional denoising

        # Convert to PIL Image for transforms
        img = Image.fromarray(img)

        if self.transform:
            img = self.transform(img)

        return img, label

## Training & Validation Transforms

- Training: Random rotation and horizontal flip for augmentation  
- Validation: Only resizing and normalization  
- Normalization values are same as ImageNet (used by pretrained ResNet)

In [7]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

val_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

## Load Dataset using Custom Class

- Handles OpenCV reading + PIL conversion  
- Applies transforms for training and validation

In [8]:
full_dataset = PlantVillageDataset(root_dir=data_dir, transform=train_transforms)

num_classes = len(full_dataset.classes)
class_names = full_dataset.classes

print("Number of classes:", num_classes)
print("Classes:", class_names)

Number of classes: 16
Classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'PlantVillage', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']
